In [1]:

# Kaggle part 
!pip install kaggle
from google.colab import files
uploaded = files.upload()  # pls add the kaggle json file to get the dataset @ datathon team
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c ds3-datathon-traffic-signs
!unzip -q ds3-datathon-traffic-signs.zip -d /content/traffic_signs
!ls /content/traffic_signs
# the necessary libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os
# citations : Towards datascience and analytics vidhya and data augmentation links attached in the slide deck for reference
#  the function to load data  
csv_path = '/content/traffic_signs/traffic_signs/traffic_train.csv'
image_base_path = '/content/traffic_signs/traffic_signs/'
def load_data(csv_path, base_path, img_size=(30, 30)):
    data = []
    labels = []
    df = pd.read_csv(csv_path)
    for idx, row in df.iterrows():
        image_path = os.path.join(base_path, row['Path'])
        try:
            image = Image.open(image_path)
            image = image.resize(img_size)
            data.append(np.array(image))
            labels.append(row['ClassId'])
        except IOError as e:
            print(f"Error loading image: {image_path} - {e}")
    data = np.array(data)
    labels = np.array(labels)
    return data, labels



# Load and preprocess data by calling teh above function
data, labels = load_data(csv_path, image_base_path)

# Splitting training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Converting the labels into one hot encoding 
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

# Data Augmentation to recognise the different orientation of the image 
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    fill_mode="nearest")

# CNN model
model = Sequential()

# First Convolutional Block with the bigger kernel/matrix
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

# Second Convolutional Block with smaller kernel/matrix to observe the details 
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

# Flattening and Dense Layers
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax')) # for multiclass classification 43 corresponds to number of classes 

# optimizing and compiling the model using standard values
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

# Callbacks to prevent overfitting issures , using standard values 
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# Training the model 
epochs = 25  
batch_size = 32

history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(X_train) // batch_size,
                    epochs=epochs,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stopping, reduce_lr])

#Saving the model 
model.save("improved_traffic_classifier.h5")


Saving kaggle.json to kaggle.json
 97% 299M/308M [00:02<00:00, 133MB/s] 
100% 308M/308M [00:02<00:00, 126MB/s]
traffic_signs


Epoch 1/25
980/980 [==============================] - 181s 182ms/step - loss: 1.8890 - accuracy: 0.4904 - val_loss: 0.4817 - val_accuracy: 0.8625 - lr: 0.0010
Epoch 2/25
980/980 [==============================] - 178s 181ms/step - loss: 0.5616 - accuracy: 0.8282 - val_loss: 0.0707 - val_accuracy: 0.9814 - lr: 0.0010
Epoch 3/25
980/980 [==============================] - 182s 185ms/step - loss: 0.3135 - accuracy: 0.9052 - val_loss: 0.0353 - val_accuracy: 0.9892 - lr: 0.0010
Epoch 4/25
980/980 [==============================] - 174s 177ms/step - loss: 0.2236 - accuracy: 0.9325 - val_loss: 0.0297 - val_accuracy: 0.9921 - lr: 0.0010
Epoch 5/25
980/980 [==============================] - 173s 176ms/step - loss: 0.1755 - accuracy: 0.9466 - val_loss: 0.0372 - val_accuracy: 0.9883 - lr: 0.0010
Epoch 6/25
980/980 [==============================] - 172s 175ms/step - loss: 0.1428 - accuracy: 0.9563 - val_loss: 0.0186 - val_accuracy: 0.9940 - lr: 0.0010
Epoch 7/25
980/980 [==========================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
# Function to load test data similar to the load data for training 
def load_test_data(csv_path, base_path, img_size=(30, 30)):
    data = []
    image_paths = []  
    df = pd.read_csv(csv_path)
    for idx, row in df.iterrows():
        image_path = os.path.join(base_path, str(row['Path']))  
        image = Image.open(image_path)
        image = image.resize(img_size)
        image = np.array(image)
        data.append(image)
        image_paths.append(image_path)
    return np.array(data), image_paths


test_csv_path = '/content/traffic_signs/traffic_signs/traffic_test.csv'
test_image_base_path = '/content/traffic_signs/traffic_signs/'


test_data, test_image_paths = load_test_data(test_csv_path, test_image_base_path)


# Predict classes for the test data by using the model 
test_predictions = model.predict(test_data)
predicted_classes = np.argmax(test_predictions, axis=1)

# Create a submission DataFrame to convert to csv
submission_df = pd.DataFrame({
    'Id': range(0, len(predicted_classes)),  # Creates a column for Ids starting from 1 to the number of predictions
    'ClassId': predicted_classes
})

# Save the submission 
submission_file_path = '/content/traffic_signs/traffic_submission.csv'
submission_df.to_csv(submission_file_path, index=False)
print(f'Submission file saved to: {submission_file_path}')

# Optionally, download the submission file from colab :)
from google.colab import files
files.download(submission_file_path)


395/395 [==============================] - 16s 40ms/step
Submission file saved to: /content/traffic_signs/traffic_submission.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>